**Keras tuner decides number of hidden layers and Nurons in Nural Network**

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 97 kB 3.2 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch # this is going to decide how many hidden layers and how many number of nurons to use in each hideen layer

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """


In [ ]:
df= pd.read_csv('/content/drive/MyDrive/ML projects/Real_Combine.csv')

In [ ]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df.dropna()

In [ ]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
1,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
3,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
5,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
7,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
9,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [ ]:
X = df.iloc[:,:-1]
y= df.iloc[:,-1]

In [ ]:
y

1       219.720833
3       182.187500
5       154.037500
7       223.208333
9       200.645833
           ...    
2177    288.416667
2179    256.833333
2181    169.000000
2183    186.041667
2185    185.583333
Name: PM 2.5, Length: 1092, dtype: float64

## what are the hyperparameters here

* How many number of hidden layers we should have
* How many no of nurons in each hidden layers we should have
* and learning rate 



In [ ]:
# random search will search best possible values required on the basis of given below range
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):                # this for loop says use any number of hideen layers between 2,20 
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,      # and this min_value says min number of nurons in each hidden layer are 32 and maximum arre 512
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear')) # here we are adding output layer that will be only one as it is regression problem statement and usually linear activation function is used for regression problems
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), # learning rates hyperparameter
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

  

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,          
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [ ]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 09s]
val_mean_absolute_error: 64.56280517578125

Best val_mean_absolute_error So Far: 60.666768391927086
Total elapsed time: 00h 00m 43s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in project/Air Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 6
units_0: 32
units_1: 288
learning_rate: 0.001
units_2: 448
units_3: 224
units_4: 288
units_5: 288
units_6: 64
units_7: 224
units_8: 256
units_9: 480
units_10: 256
Score: 60.666768391927086
Trial summary
Hyperparameters:
num_layers: 9
units_0: 416
units_1: 224
learning_rate: 0.001
units_2: 96
units_3: 512
units_4: 224
units_5: 128
units_6: 64
units_7: 288
units_8: 384
units_9: 160
units_10: 224
Score: 61.980542500813804
Trial summary
Hyperparameters:
num_layers: 11
units_0: 256
units_1: 320
learning_rate: 0.0001
units_2: 192
units_3: 256
units_4: 448
units_5: 384
units_6: 480
units_7: 192
units_8: 160
units_9: 352
units_10: 32
Score: 64.40200297037761
Trial summary
Hyperparameters:
num_layers: 9
units_0: 448
units_1: 192
learning_rate: 0.0001
units_2: 96
units_3: 128
units_4: 128
units_5: 160
units_6: 416
unit